In [1]:
import os
import sys

import numpy as np
import pandas as pd
import dotenv
import mlflow
from autogluon.common import space
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import plotly.graph_objects as go
from huggingface_hub import login

sys.path.append("../..")

from utils import calculate_sklearn_metrics, TrainingConfig
from utils.plotting import plot_forecasts_val_test

dotenv.load_dotenv("../../.env")

token = os.environ["HF_TOKEN"]
login(token=token)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("rosstat_forecasting");

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
data_dir = '../../../data/rosstat/processed'

train_df = pd.read_csv(os.path.join(data_dir, 'train/data.csv'))
val_df = pd.read_csv(os.path.join(data_dir, 'val/data.csv'))
test_df = pd.read_csv(os.path.join(data_dir, 'test/data.csv'))

print(f"Обучающая выборка: {train_df.shape[0]} строк")
print(f"Валидационная выборка: {val_df.shape[0]} строк")
print(f"Тестовая выборка: {test_df.shape[0]} строк")

Обучающая выборка: 4140 строк
Валидационная выборка: 828 строк
Тестовая выборка: 828 строк


In [3]:
train_data = TimeSeriesDataFrame.from_data_frame(
    train_df.rename(columns={"nominal_wage": "target"}),
    id_column="code",
    timestamp_column="date",
)
val_data = TimeSeriesDataFrame.from_data_frame(
    val_df.rename(columns={"nominal_wage": "target"}),
    id_column="code",
    timestamp_column="date",
)
test_data = TimeSeriesDataFrame.from_data_frame(
    test_df.rename(columns={"nominal_wage": "target"}),
    id_column="code",
    timestamp_column="date",
)

In [4]:
config = TrainingConfig(
    prediction_length=2,  # полгода
    artifact_path="../models/auto_ml_hpo",
)

predictor = TimeSeriesPredictor(
    prediction_length=config.prediction_length, path=config.artifact_path, freq="MS"
).fit(
    train_data=train_data,
    tuning_data=val_data,
    verbosity=4,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_base",
                "fine_tune": True,
                "ag_args": {"name_suffix": "FineTuned"},
                "context_length": space.Categorical(2048, 512, 1024),
                "fine_tune_lr": space.Real(1e-6, 1e-4),
                "fine_tune_steps": space.Categorical(1000, 500, 1500),
                "fine_tune_shuffle_buffer_size": space.Categorical(10000, 5000, 10000, None),
                "fine_tune_eval_max_items": None,
            },
        ],
    },
    hyperparameter_tune_kwargs={
        "num_trials": 16,
        "scheduler": "local",
        "searcher": "bayes",
    },
    enable_ensemble=False,
)

Beginning AutoGluon training...
AutoGluon will save models to '/home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo'
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.12.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #61~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Apr 15 17:03:15 UTC 2
CPU Count:          12
GPU Count:          1
Memory Avail:       20.45 GB / 30.95 GB (66.1%)
Disk Space Avail:   125.05 GB / 233.67 GB (53.5%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameter_tune_kwargs': {'num_trials': 16,
                                'scheduler': 'local',
                                'searcher': 'bayes'},
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'FineTuned'},
                                  'context_length': Categorical[2048, 512, 1024],
                                  'fine_tune': True,
               

  0%|          | 0/16 [00:00<?, ?it/s]

	Saving fine-tuned model to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T1/fine-tuned-ckpt
Removing transformers_logs directory /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T1/transformers_logs
	Hyperparameter tune run: ChronosFineTuned[bolt_base]/T1
		-0.1247      = Validation score (-WQL)
		493.023 s    = Training runtime
		0.291   s    = Training runtime
	Saving fine-tuned model to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T2/fine-tuned-ckpt
Removing transformers_logs directory /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T2/transformers_logs
	Hyperparameter tune run: ChronosFineTuned[bolt_base]/T2
		-0.1035      = Validation score (-WQL)
		243.800 s    = Training runtime
		0.296   s    = Training runtime
	S

In [5]:
leaderboard = predictor.leaderboard(
    test_data,
    extra_metrics=['MASE', 'MAPE', 'MSE', 'MAE', 'SQL'],
)
leaderboard.rename(columns={'score_test': 'WQL_test', 'score_val': 'WQL_val'}, inplace=True)
leaderboard

Generating leaderboard for all models trained
Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).
Prediction order: ['ChronosFineTuned[bolt_base]/T13', 'ChronosFineTuned[bolt_base]/T12', 'ChronosFineTuned[bolt_base]/T9', 'ChronosFineTuned[bolt_base]/T15', 'ChronosFineTuned[bolt_base]/T16', 'ChronosFineTuned[bolt_base]/T3', 'ChronosFineTuned[bolt_base]/T10', 'ChronosFineTuned[bolt_base]/T8', 'ChronosFineTuned[bolt_base]/T11', 'ChronosFineTuned[bolt_base]/T2', 'ChronosFineTuned[bolt_base]/T6', 'ChronosFineTuned[bolt_base]/T4', 'ChronosFineTuned[bolt_base]/T1', 'ChronosFineTuned[bolt_base]/T7', 'ChronosFineTuned[bolt_base]/T5', 'ChronosFineTuned[bolt_base]/T14']
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T13/fine-tuned-ckpt
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/

,model,WQL_test,WQL_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order,MASE,MAPE,MSE,MAE,SQL
0,ChronosFineTuned[bolt_base]/T5,-0.065714,-0.072451,1.734690,0.148123,266.938163,5,-7511.966587,-0.066741,-1.599482e+08,-7511.966587,-6162.580565
1,ChronosFineTuned[bolt_base]/T14,-0.069018,-0.105230,0.695978,0.287550,504.053381,14,-7899.381094,-0.068626,-1.795308e+08,-7899.381094,-6472.407072
2,ChronosFineTuned[bolt_base]/T6,-0.071828,-0.089341,0.464083,0.080836,162.219328,6,-7948.628343,-0.067786,-1.971385e+08,-7948.628343,-6735.939646
3,ChronosFineTuned[bolt_base]/T9,-0.073364,-0.100724,0.650778,0.288878,711.870040,9,-8118.888910,-0.069755,-2.049869e+08,-8118.888910,-6879.969736
4,ChronosFineTuned[bolt_base]/T12,-0.073693,-0.087158,0.528136,0.179976,265.373652,12,-8236.581830,-0.071375,-1.969192e+08,-8236.581830,-6910.837152
5,ChronosFineTuned[bolt_base]/T10,-0.074878,-0.097934,0.655411,0.300767,475.833919,10,-8444.762053,-0.072355,-2.078698e+08,-8444.762053,-7021.908123
6,ChronosFineTuned[bolt_base]/T11,-0.076727,-0.097706,0.526826,0.145700,385.849385,11,-8556.474881,-0.072166,-2.169799e+08,-8556.474881,-7195.341295
7,ChronosFineTuned[bolt_base]/T2,-0.079984,-0.103497,0.894876,0.295617,243.800074,2,-9029.649564,-0.075847,-2.350057e+08,-9029.649564,-7500.755350
8,ChronosFineTuned[bolt_base]/T13,-0.080553,-0.111808,0.756791,0.308975,742.779677,13,-8993.555681,-0.077692,-2.233117e+08,-8993.555681,-7554.149544
9,ChronosFineTuned[bolt_base]/T3,-0.083539,-0.112817,1.463511,0.079632,238.970931,3,-9294.609117,-0.078312,-2.525424e+08,-9294.609117,-7834.106300


In [6]:
def extract_specific_rows_from_indexed_data(data, start_row: int, end_row: int):
    rows_to_extract = np.arange(start_row, end_row)
    unique_ids = data.index.get_level_values('item_id').unique()
        
    selected_data = []

    for item_id in unique_ids:
        item_data = data.loc[[item_id]]
        
        selected_rows = item_data.iloc[rows_to_extract]
        selected_data.append(selected_rows)

    result = pd.concat(selected_data)

    return result

k = 10

top_k_models = leaderboard.sort_values(['SQL'], ascending=False).head(k)['model'].tolist()
window_size = config.prediction_length
test_length = test_df['code'].value_counts().iloc[0]
max_iterations = (test_length + window_size - 1) // window_size# - 1 ещё -1 из-за known_covariates

current_data = train_data.copy()
all_val_models_predictions = {}

for i in range(max_iterations):
    start_idx = i * window_size
    end_idx = start_idx + window_size
    
    for model_name in top_k_models:
        if model_name not in all_val_models_predictions:
            all_val_models_predictions[model_name] = []
        
        # future_covariates = test_data[start_idx:start_idx + config.prediction_length][known_covariates_names]
        # prediction_covariates = pd.concat([current_data[known_covariates_names], future_covariates])
        
        predictions = predictor.predict(current_data, 
                                       model=model_name,)
                                       # known_covariates=prediction_covariates)
                                       
        all_val_models_predictions[model_name].append(predictions)
        
    current_data = pd.concat([current_data, extract_specific_rows_from_indexed_data(val_data, start_idx, end_idx)])

test_df_shape = test_df.shape[0]
all_val_models_predictions = {k: pd.concat(v)[:test_df_shape] for k, v in all_val_models_predictions.items()}

current_data = val_data.copy()
all_test_models_predictions = {}

for i in range(max_iterations):
    start_idx = i * window_size
    end_idx = start_idx + window_size
    
    for model_name in top_k_models:
        if model_name not in all_test_models_predictions:
            all_test_models_predictions[model_name] = []
        
        # future_covariates = test_data[start_idx:start_idx + config.prediction_length][known_covariates_names]
        # prediction_covariates = pd.concat([current_data[known_covariates_names], future_covariates])
        
        predictions = predictor.predict(current_data, 
                                       model=model_name,)
                                       # known_covariates=prediction_covariates)
                                       
        all_test_models_predictions[model_name].append(predictions)
        
    current_data = pd.concat([current_data, extract_specific_rows_from_indexed_data(test_data, start_idx, end_idx)])

test_df_shape = test_df.shape[0]
all_test_models_predictions = {k: pd.concat(v)[:test_df_shape] for k, v in all_test_models_predictions.items()}

Prediction order: {'ChronosFineTuned[bolt_base]/T5'}
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T5/fine-tuned-ckpt


Cached predictions saved to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/cached_predictions.pkl
Prediction order: {'ChronosFineTuned[bolt_base]/T14'}
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T14/fine-tuned-ckpt
Cached predictions saved to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/cached_predictions.pkl
Prediction order: {'ChronosFineTuned[bolt_base]/T6'}
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T6/fine-tuned-ckpt
Cached predictions saved to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/cached_predictions.pkl
Prediction order: {'ChronosFineTuned[bolt_base]/T9'}
	Fine-tuned checkpoint exists, setting model_path to /ho

In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_forecasts_val_test(
    val_df: pd.DataFrame,
    test_df: pd.DataFrame,
    val_predictions: pd.DataFrame,
    test_predictions: dict[str, pd.DataFrame],
    start_date: str | None = None,
    end_date: str | None = None,
    height: int = 300,
    width: int = 1400,
    item_id: str | None = None,
):
    model_names = list(test_predictions.keys())
    n_models = len(model_names)

    if n_models == 1:
        rows, cols = 1, 1
    else:
        rows = int(np.ceil(n_models / 2))
        cols = 2
    
    filtered_val_df = val_df.copy()
    filtered_test_df = test_df.copy()
    
    if start_date is not None and end_date is not None:        
        test_mask = (filtered_test_df["timestamp"] >= start_date) & (filtered_test_df["timestamp"] <= end_date)
        filtered_test_df = filtered_test_df[test_mask]
    
    fig = make_subplots(
        rows=rows, cols=cols, subplot_titles=model_names, vertical_spacing=0.1
    )
    
    for i, model_name in enumerate(model_names):
        row = i // cols + 1
        col = i % cols + 1

        model_val_pred = val_predictions[model_name].copy()
        if item_id is not None:
            model_val_pred = model_val_pred.loc[item_id]

        filtered_val_mean = model_val_pred["mean"].values
        filtered_val_upper = model_val_pred["0.9"].values if "0.9" in model_val_pred else None
        filtered_val_lower = model_val_pred["0.1"].values if "0.1" in model_val_pred else None
        
        model_test_pred = test_predictions[model_name]
        if item_id is not None:
            model_test_pred = model_test_pred.loc[item_id]
        
        if start_date is not None and end_date is not None:
            test_time_mask = (test_df["timestamp"] >= start_date) & (test_df["timestamp"] <= end_date)
            
            filtered_test_mean = model_test_pred["mean"].values[test_time_mask]
            filtered_test_upper = model_test_pred["0.9"].values[test_time_mask] if "0.9" in model_test_pred else None
            filtered_test_lower = model_test_pred["0.1"].values[test_time_mask] if "0.1" in model_test_pred else None
            
        else:
            filtered_test_mean = model_test_pred["mean"]
            filtered_test_upper = model_test_pred["0.9"] if "0.9" in model_test_pred else None
            filtered_test_lower = model_test_pred["0.1"] if "0.1" in model_test_pred else None
        
        fig.add_trace(
            go.Scatter(
                x=filtered_val_df["timestamp"],
                y=filtered_val_df["target"],
                name="Validation (actual)",
                line=dict(color="blue"),
            ),
            row=row,
            col=col,
        )
        
        fig.add_trace(
            go.Scatter(
                x=filtered_val_df["timestamp"],
                y=filtered_val_mean,
                name="Validation (predicted)",
                line=dict(color="purple", dash="dot"),
            ),
            row=row,
            col=col,
        )
        
        fig.add_trace(
            go.Scatter(
                x=filtered_test_df["timestamp"],
                y=filtered_test_df["target"],
                name="Test (actual)",
                line=dict(color="#50C878"),
            ),
            row=row,
            col=col,
        )
        
        fig.add_trace(
            go.Scatter(
                x=filtered_test_df["timestamp"],
                y=filtered_test_mean,
                name=f"{model_name} Test (predicted)",
                line=dict(color="#D70040", dash="dot"),
            ),
            row=row,
            col=col,
        )

        fig.add_trace(
                go.Scatter(
                    x=filtered_val_df["timestamp"],
                    y=filtered_val_upper,
                    mode="lines",
                    line=dict(width=0),
                    showlegend=False,
                ),
                row=row,
                col=col,
            )
        fig.add_trace(
            go.Scatter(
                x=filtered_val_df["timestamp"],
                y=filtered_val_lower,
                mode="lines",
                fill="tonexty",
                fillcolor="rgba(128, 0, 128, 0.6)",
                line=dict(width=0),
                name=f"{model_name} CI (0.1-0.9)",
            ),
            row=row,
            col=col,
        )
        
        if filtered_test_upper is not None and filtered_test_lower is not None:
            fig.add_trace(
                go.Scatter(
                    x=filtered_test_df["timestamp"],
                    y=filtered_test_upper,
                    mode="lines",
                    line=dict(width=0),
                    showlegend=False,
                ),
                row=row,
                col=col,
            )
            fig.add_trace(
                go.Scatter(
                    x=filtered_test_df["timestamp"],
                    y=filtered_test_lower,
                    mode="lines",
                    fill="tonexty",
                    fillcolor="rgba(255, 127, 14, 0.6)",
                    line=dict(width=0),
                    name=f"{model_name} CI (0.1-0.9)",
                ),
                row=row,
                col=col,
            )
    
    fig.update_layout(
        title="Forecasts with Validation and Test Data",
        template="plotly_white",
        height=height * rows,
        width=width,
        showlegend=True,
    )
    
    for i in range(rows * cols):
        row = i // cols + 1
        col = i % cols + 1
        
        fig.update_xaxes(
            title_text="Date",
            row=row,
            col=col,
        )
        fig.update_yaxes(
            title_text="National Demand",
            row=row,
            col=col,
        )
    
    fig.show()

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
from utils.plotting import plot_forecasts

date_col = pd.to_datetime(test_df["date"])
min_date = date_col.min().date()
max_date = date_col.max().date()
height = 400
width = 1200

start_date_picker = widgets.DatePicker(
    description="Start date:", disabled=False, value=min_date
)

end_date_picker = widgets.DatePicker(
    description="End date:", disabled=False, value=max_date
)

output_area = widgets.Output()


def on_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        start_date = datetime.datetime.combine(
            start_date_picker.value, datetime.datetime.min.time()
        )
        end_date = datetime.datetime.combine(
            end_date_picker.value, datetime.datetime.min.time()
        )
        plot_forecasts_val_test(
            val_df=val_df_,
            test_df=test_df_,
            val_predictions=all_val_models_predictions_,
            test_predictions=all_test_models_predictions,
            start_date=start_date,
            end_date=end_date,
            height=height,
            width=width,
            item_id=item_id,
        )


plot_button = widgets.Button(description="Plot Forecasts")
plot_button.on_click(on_button_clicked)

controls = widgets.VBox(
    [widgets.HBox([start_date_picker, end_date_picker]), plot_button]
)

display(controls, output_area)

# item_id='82.91'
# item_id='01.11.39'
item_id='82.99'
item_id = 81

val_df_ = val_df.rename(columns={'date': 'timestamp', "nominal_wage": "target"})[['code', 'timestamp', "target"]]
val_df_ = val_df_[val_df_['code'].eq(item_id)].reset_index(drop=True)
val_df_['timestamp'] = pd.to_datetime(val_df_['timestamp'])

test_df_ = test_df.rename(columns={'date': 'timestamp', "nominal_wage": "target"})[['code', 'timestamp', "target"]]
test_df_ = test_df_[test_df_['code'].eq(item_id)].reset_index(drop=True)
test_df_['timestamp'] = pd.to_datetime(test_df_['timestamp'])

val_df_ = pd.concat([val_df_, test_df_.iloc[[0]]])

all_val_models_predictions_ = all_val_models_predictions.copy()
for model in all_val_models_predictions_.keys():
    all_val_models_predictions_[model] = pd.concat([all_val_models_predictions_[model], all_test_models_predictions[model].loc[[item_id]].iloc[[0]]])

with output_area:
    plot_forecasts_val_test(
        val_df=val_df_,
        test_df=test_df_,
        val_predictions=all_val_models_predictions_,
        test_predictions=all_test_models_predictions,
        height=height,
        width=width,
        item_id=item_id,
    )

Output()

In [9]:
all_codes = test_df['code'].unique()

all_models_metrics = {}

for model in all_test_models_predictions.keys():
    metrics_df = []
    for code in all_codes:
        pred_df = pd.concat([
            all_test_models_predictions[model]
            .loc[code][["0.1", "0.5", "0.9"]]
            .reset_index(drop=True),
            test_df[test_df["code"].eq(code)][["nominal_wage"]].reset_index(drop=True),
        ], axis=1)
        pred_df = pd.DataFrame(pred_df)

        metrics_df.append(calculate_sklearn_metrics(pred_df, target_column='nominal_wage'))

    metrics_dict = pd.DataFrame(metrics_df).mean().to_dict()

    all_models_metrics[model] = metrics_dict

In [18]:
min_MSE = float('inf')
best_model = ''

for k, v in all_models_metrics.items():
    if (temp_mse := v['MAE']) < min_MSE:
        min_MSE = temp_mse
        best_model = k

best_model

'ChronosFineTuned[bolt_base]/T2'

In [19]:
hpo_results = predictor.fit_summary()
model_hyperparams = hpo_results['model_hyperparams'][best_model]

best_model_metrics = dict(best_model=all_models_metrics[best_model])
best_model_metrics

	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T1/fine-tuned-ckpt
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T2/fine-tuned-ckpt
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T3/fine-tuned-ckpt
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T4/fine-tuned-ckpt
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/projects/time_series_analysis/code_dir/rosstat/models/auto_ml_hpo/models/ChronosFineTuned[bolt_base]/T5/fine-tuned-ckpt
	Fine-tuned checkpoint exists, setting model_path to /home/nikita/proj

****************** Summary of fit() ******************
Estimated performance of each model:
                              model  score_val  pred_time_val  \
0    ChronosFineTuned[bolt_base]/T5  -0.072451       0.148123   
1   ChronosFineTuned[bolt_base]/T12  -0.087158       0.179976   
2    ChronosFineTuned[bolt_base]/T6  -0.089341       0.080836   
3   ChronosFineTuned[bolt_base]/T11  -0.097706       0.145700   
4   ChronosFineTuned[bolt_base]/T10  -0.097934       0.300767   
5   ChronosFineTuned[bolt_base]/T16  -0.098342       0.079850   
6    ChronosFineTuned[bolt_base]/T9  -0.100724       0.288878   
7    ChronosFineTuned[bolt_base]/T8  -0.101642       0.168128   
8    ChronosFineTuned[bolt_base]/T4  -0.102901       0.293809   
9    ChronosFineTuned[bolt_base]/T2  -0.103497       0.295617   
10  ChronosFineTuned[bolt_base]/T14  -0.105230       0.287550   
11  ChronosFineTuned[bolt_base]/T13  -0.111808       0.308975   
12  ChronosFineTuned[bolt_base]/T15  -0.112589       0.080879  

{'best_model': {'MSE': 39255938.397107,
  'MAE': 3160.4115819368953,
  'MAPE': 3.4741186104910167,
  'MASE': 0.6093397862056688,
  'SQL': 1050.2688319306064}}

In [20]:
prefix = 'AutoGluon_Chronos_HPO'

for k, metrics_ in best_model_metrics.items():
    k = best_model.rstrip('/T0123456789') + '_HPO'
    run_name = f"{k}_{prefix}"

    with mlflow.start_run(run_name=run_name):
        mlflow.log_metrics(metrics_)
        mlflow.log_param("model_name", model_name)
        mlflow.log_params(model_hyperparams)

        mlflow.set_tag("prefix", prefix)

🏃 View run ChronosFineTuned[bolt_base]_HPO_AutoGluon_Chronos_HPO at: http://127.0.0.1:5000/#/experiments/169882278836627198/runs/153abfdc3abf4fd786e3269031cda748
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/169882278836627198
